In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_pic = pd.read_csv('df_pic.csv')
df_prod = pd.read_csv('df_prod.csv')

In [ ]:
df_pic['ADMITTED_DT'] = pd.to_datetime(df_pic['ADMITTED_DT'], format='%m/%d/%Y %H:%M')
df_pic['DISCHARGED_DT'] = pd.to_datetime(df_pic['DISCHARGED_DT'], format='%m/%d/%Y %H:%M')

admitted_dates_df = df_pic[['BOOKCASENUMBER_HASHED', 'ADMITTED_DT', 'SEVERITY']]
discharge_dates_df = df_pic[['BOOKCASENUMBER_HASHED', 'DISCHARGED_DT']]

In [ ]:
df_prod['DOC_DEPARTURE_DATETIME'] = pd.to_datetime(df_prod['DOC_DEPARTURE_DATETIME'], format='%m/%d/%Y %H:%M')
court_dates_df = df_prod[['BOOKCASENUMBER_HASHED', 'DOC_DEPARTURE_DATETIME', 'COUNTY']]
court_dates_df = court_dates_df.sort_values(by=['BOOKCASENUMBER_HASHED', 'DOC_DEPARTURE_DATETIME']).reset_index(drop=True)

In [ ]:
# Merge df_court_dates with df_admit_dates
merged_df = pd.merge(court_dates_df, admitted_dates_df, on='BOOKCASENUMBER_HASHED', how='left')

# Merge the result with df_discharge_dates
merged_df = pd.merge(merged_df, discharge_dates_df, on='BOOKCASENUMBER_HASHED', how='left')
merged_df = merged_df.dropna(subset = ['ADMITTED_DT'])
merged_df.head(50)

In [ ]:
# Drop duplicates to keep only the first court date for each inmate
first_court_dates_df = merged_df.drop_duplicates(subset='BOOKCASENUMBER_HASHED', keep='first')

# Calculate the time difference between 'admit_date' and 'court_date'
first_court_dates_df['time_to_first_court'] = first_court_dates_df['DOC_DEPARTURE_DATETIME'] - first_court_dates_df['ADMITTED_DT']
first_court_dates_df.head()

In [ ]:
# Drop duplicates to keep only the last court date for each inmate
final_court_dates_df = merged_df.drop_duplicates(subset='BOOKCASENUMBER_HASHED', keep='last')

# Calculate the time difference between 'discharge_date' and 'final_court_date'
final_court_dates_df['time_since_last_court'] = final_court_dates_df['DISCHARGED_DT'] - final_court_dates_df['DOC_DEPARTURE_DATETIME']
final_court_dates_df.head()

In [ ]:
#To find the number of court dates for each PIC
court_date_counts = merged_df.groupby('BOOKCASENUMBER_HASHED').size().reset_index(name='court_date_count')

In [ ]:
# Calculate time differences between subsequent court dates for each inmate id
merged_df['court_date_diffs'] = merged_df.groupby('BOOKCASENUMBER_HASHED')['DOC_DEPARTURE_DATETIME'].diff()
subsequent_court_times_df = merged_df.groupby('BOOKCASENUMBER_HASHED').agg({
    'COUNTY': 'first',
    'ADMITTED_DT': 'first',
    'SEVERITY': 'first',
    'DISCHARGED_DT': 'first',
    'court_date_diffs': lambda x: x.dropna().tolist()  # Aggregate differences into lists
}).reset_index()

# Function to remove zero timedelta values
def remove_zero_timedeltas(list):
    return [td for td in list if td != pd.Timedelta(0)]

subsequent_court_times_df['court_date_diffs'] = subsequent_court_times_df['court_date_diffs'].apply(remove_zero_timedeltas)
pd.set_option('display.max_colwidth', None)

# Function to calculate the number of court dates
def count_court_dates(diffs):
    if not diffs:
        return 1  # Only 1 court date if the list is empty
    return len(diffs) + 1

# Apply the function to the 'court_date_diffs' column
subsequent_court_times_df['number_of_court_dates'] = subsequent_court_times_df['court_date_diffs'].apply(count_court_dates)

# Plotting the distribution of frequency of court dates
plt.figure(figsize=(14, 6))
plt.hist(subsequent_court_times_df['number_of_court_dates'], bins=range(0, max(subsequent_court_times_df['number_of_court_dates']) + 2), edgecolor='black')
plt.xlabel('Number of Court Dates')
plt.ylabel('Frequency')
plt.title('Distribution of Number of Court Dates')
plt.xticks(range(0, max(subsequent_court_times_df['number_of_court_dates']) + 1))
plt.tight_layout()
plt.show()

In [ ]:
def avg_court_date_diff(list):
    if len(list) == 0:
        return None
    return sum(list, pd.Timedelta(0)) / len(list)

# Calculate the average court date difference for each inmate
subsequent_court_times_df['avg_court_date_diff'] = subsequent_court_times_df['court_date_diffs'].apply(lambda x: avg_court_date_diff(x))

In [ ]:
subsequent_court_times_df = subsequent_court_times_df.dropna(subset=['avg_court_date_diff'])

def timedelta_to_decimal_days(td):
    total_seconds = td.total_seconds()
    decimal_days = total_seconds / (24 * 60 * 60)  # Convert seconds to days
    return decimal_days

subsequent_court_times_df['avg_court_date_diff_decimal'] = subsequent_court_times_df['avg_court_date_diff'].apply(timedelta_to_decimal_days)

county_avg = subsequent_court_times_df.groupby('COUNTY')['avg_court_date_diff_decimal'].mean()
top_charge_avg = subsequent_court_times_df.groupby('SEVERITY')['avg_court_date_diff_decimal'].mean()

# Convert to a DataFrame for plotting
county_avg_df = county_avg.reset_index()
top_charge_avg_df = top_charge_avg.reset_index()

# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

ax1.bar(county_avg_df['COUNTY'], county_avg_df['avg_court_date_diff_decimal'])
ax1.set_xlabel('County')
ax1.set_ylabel('Average Subsequent Court Date Time Difference (Days)')
ax1.set_title('Average Subsequent Court Date Time Difference by County')

ax2.bar(top_charge_avg_df['SEVERITY'], top_charge_avg_df['avg_court_date_diff_decimal'])
ax2.set_xlabel('Severity')
ax2.set_ylabel('Average Subsequent Court Date Time Difference (Days)')
ax2.set_title('Average Subsequent Court Date Time Difference by Top Charge')
plt.tight_layout()
plt.show()

In [ ]:
first_court_dates_df['avg_time_to_first_court'] = first_court_dates_df['time_to_first_court'].apply(timedelta_to_decimal_days)
first_court_county_avg = first_court_dates_df.groupby('COUNTY')['avg_time_to_first_court'].mean()
first_court_top_charge_avg = first_court_dates_df.groupby('SEVERITY')['avg_time_to_first_court'].mean()

first_court_county_avg_df = first_court_county_avg.reset_index()
first_court_top_charge_avg_df = first_court_top_charge_avg.reset_index()

# Plotting
fig, (ax3, ax4) = plt.subplots(1, 2, figsize=(14, 7))

ax3.bar(first_court_county_avg_df['COUNTY'], first_court_county_avg_df['avg_time_to_first_court'])
ax3.set_xlabel('County')
ax3.set_ylabel('Average Time to First Court (Days)')
ax3.set_title('Average Time to First Court by County')

ax4.bar(first_court_top_charge_avg_df['SEVERITY'], first_court_top_charge_avg_df['avg_time_to_first_court'])
ax4.set_xlabel('Severity')
ax4.set_ylabel('Average Time to First Court (Days)')
ax4.set_title('Average Time to First Court by Top Charge')
plt.tight_layout()
plt.show()

In [ ]:
final_court_dates_df['avg_time_since_last_court'] = final_court_dates_df['time_since_last_court'].apply(timedelta_to_decimal_days)
final_court_county_avg = final_court_dates_df.groupby('COUNTY')['avg_time_since_last_court'].mean()
final_court_top_charge_avg = final_court_dates_df.groupby('SEVERITY')['avg_time_since_last_court'].mean()

final_court_county_avg_df = final_court_county_avg.reset_index()
final_court_top_charge_avg_df = final_court_top_charge_avg.reset_index()

# Plotting
fig, (ax5, ax6) = plt.subplots(1, 2, figsize=(14, 7))

ax5.bar(final_court_county_avg_df['COUNTY'], final_court_county_avg_df['avg_time_since_last_court'])
ax5.set_xlabel('County')
ax5.set_ylabel('Average Time Since Last Court (Days)')
ax5.set_title('Average Time Since Last Court by County')

ax6.bar(final_court_top_charge_avg_df['SEVERITY'], final_court_top_charge_avg_df['avg_time_since_last_court'])
ax6.set_xlabel('Severity')
ax6.set_ylabel('Average Time Since Last Court (Days)')
ax6.set_title('Average Time Since Last Court by Top Charge')
plt.tight_layout()
plt.show()